## Call WOfS using bounding boxes
This notebook merges gauge data with satellite data for multiple reservoirs at a time (in the 00_Library diresctory). It also makes the depth to surface area relatiosnhips for the reservoirs. It's my main notebook for this project and closest to what I have as the final product. Bounding boxes are used to get the exact extents of reservoirs for the depth-to-surface area relationship. It's an automated way to call the right satellite data accurately. This notebook uses a lot of memory and will crash if it hits the limit of 15GB. When you run this, make sure you don't have any other notebooks running at the same time (click the cirlce with a square icon on the left side of the creen and shutdown any other notebooks you have running). 

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import csv
import rasterio.crs
from tqdm.auto import tqdm #this one is a loading bar, it's cool to add loading bars to loops
from pandas import DataFrame
import geopandas as gpd
import matplotlib.gridspec as gs
import matplotlib.pyplot as plt
from matplotlib import pyplot
import datacube

import sys
sys.path.append('../../Scripts')
from dea_spatialtools import xr_rasterize
from dea_datahandling import wofs_fuser #this joins wofs data across tiles correctly
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.utils import masking
from datacube.helpers import ga_pq_fuser, write_geotiff
#from digitalearthau.utils import wofs_fuser
#import DEAPlotting, DEADataHandling
import warnings
warnings.filterwarnings('ignore', module='datacube')
%load_ext autoreload
%autoreload 2

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Dask load the satellite data for all the reservoirs
'Dask load' means you only load parameters, not the actual images, which is good for linking the satellite data to the gauge data. I made a shapefile in ArcMAP that has bounding boxes of the reservoirs identified in 00_Library_reservoirs. This is what we will use for the query, so wofs knows where to make the picture for each reservoir. The following code blocks were copied from a DEA notebook called 'Open and run analysis on multiple polygons'. In this case the multiple polygons are the bounding boxes in 00_Lib_bound. 

In [2]:
gdf = gpd.read_file('00_Lib_bound/00_Lib_bound.shp')

query = {'time': ('01-01-1988', '09-12-2020')} 
         #'crs': 'EPSG:3577'}
dc = datacube.Datacube(app='dc-WOfS')

results = {} 

#tqdm is gonna make the bar. tqdm is Arabic abbreviation for 'progress'
for index, row in tqdm(gdf.iterrows(), total=len(gdf)):
    geom = geometry.Geometry(geom=row.geometry, crs=gdf.crs)
    query.update({'geopolygon': geom})
    
    wofs_albers= dc.load(product = 'wofs_albers', dask_chunks = {}, 
                         group_by='solar_day', fuse_func = wofs_fuser, **query) #wofs_fuser is important, it fixes thing on the edge of tiles
    
    poly_mask = xr_rasterize(gdf.iloc[[index]], wofs_albers)
    wofs_albers = wofs_albers.where(poly_mask, other=wofs_albers.water.nodata) #put other argument or all the data turns into 0
    
    results.update({str(row['gauge_ID']): wofs_albers}) #The handle for dictionary objects is the gauge ID

  0%|          | 0/153 [00:00<?, ?it/s]

## loop read all the csv files in 00_Library
In the previous code block we made a dictionary of the wofs data with the gauge ID as the key. We now need a dictionary of the depth data with, again, the gauge ID as the key. Then we can match them up later. 

In [3]:
#make a list of the file names so we can call them with pandas
file_list = []

directory = '00_Library'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_list.append(os.path.join(directory, filename))

#Read the gauge files twice, once to get ID and second to get the data. Append them together in a dictionary
#May as well make a list of IDs here because we will use it later
data_dict = {}        
ID_list = []
#let's use tqdm again to make a progress bar. The bar is so cool I love this module
for i in tqdm(file_list, total=len(file_list)):
    df = pd.read_csv(i, nrows=1, escapechar='#')
    column = df.iloc[:,[1]] #This is the column with the ID in it
    ID = list(column)
    ID = ID[0]
    ID = df.at[0, ID]
    ID_list.append(str(ID))
    
    data = pd.read_csv(i, error_bad_lines = False, skiprows=9, escapechar='#',
                         parse_dates=['Timestamp'], 
                         index_col=('Timestamp'),
                        date_parser=lambda x: pd.to_datetime(x.rsplit('+', 1)[0]))
    data = data.drop(columns=['Quality Code', 'Interpolation Type'])
    data_dict.update({str(ID): data}) #Now we have the gauge data, again with the gauge ID as the handle

  0%|          | 0/173 [00:00<?, ?it/s]

## Make a function that generates the depth slices of a reservoir and gets the depth to surface area relationship
This is my first time writing a function, Matthew and Bex from DEA helped me write it. There's an argument in here that says 'if there's more than 25 1m intervals, only take every second meter'. I think this makes the data better but you can change this if you want every meter (just change 25 to 1000 or something so none of the reservoirs will meet it and they can all be made as 1m intervals). There is also an argument in here which says 'only take the first 15 passes for every depth slice' in the line 'date_list = specified_level.time.values[:15]'. I put this in to make the code run faster, but the results will be less accurate. You should try to hash out the [:15] part and run it with every available pass, but it will take well over an hour and it may break the kernel. Make sure you shutdown any other notebooks to free up memory (see the icon on the left with the circle and square to shutdown other notebooks). 

In [4]:
def image_prod(ID_caller, gauge_data, wofs_albers, make_plots = False) -> 'depth slices': 
    """
    This function takes the gauge data and the wofs data,
    cloud masks the images and counts the pixels in each depth slice.
    It returns a list of all the surface areas per depth.
    
    """
    #Get the depth range and intervals
    gauge_data = gauge_data.dropna()
    depth_integers = gauge_data.astype(np.int64)
    max_depth = depth_integers.Value.max()
    min_depth = depth_integers.Value.min()
    integer_array = depth_integers.Value.unique()
    integer_list = integer_array.tolist()
    
    gauge_data_xr = gauge_data.to_xarray() #convert gauge data to xarray
    merged_data = gauge_data_xr.interp(Timestamp=wofs_albers.time) #use xarrays .interp() function to merge

    surface_area_list = []

    for i in tqdm(integer_list, leave = False):
        if len(integer_list) > 25: #If the depth range is more than 25 1m intervals, take every 2m instead of 1m
            specified_level = merged_data.where((merged_data.Value > i) & 
                                (merged_data.Value < i+2), drop=True)
        else:
            specified_level = merged_data.where((merged_data.Value > i) & 
                                (merged_data.Value < i+1), drop=True)


        date_list = specified_level.time.values[:15] #caps images at x per slice (way faster, prevents crashing)
        n_images_used = int(len(date_list))
        specified_passes = wofs_albers.sel(time=date_list).compute() #This .compute() Xarray function loads actual images
        #cloudmask (Claire Krause wrote this for me)
        #print(specified_passes.water)
        cc = masking.make_mask(specified_passes.water, cloud=True)
        ncloud_pixels = cc.sum(dim=['x', 'y'])
        # Calculate the total number of pixels per timestep
        npixels_per_slice = (specified_passes.water.shape[1] * 
                             specified_passes.water.shape[2])
        cloud_pixels_fraction = (ncloud_pixels / npixels_per_slice)
        clear_specified_passes = specified_passes.water.isel(
            time=cloud_pixels_fraction < 0.2) #has to be under 20% cloudy to pass
        wet = masking.make_mask(clear_specified_passes, wet=True).sum(dim='time')
        dry = masking.make_mask(clear_specified_passes, dry=True).sum(dim='time')
        clear = wet + dry
        frequency = wet / clear
        frequency = frequency.fillna(0)  

        #Get area from the satellite data
        #get the frequency array
        frequency_array = frequency.values
        n_images_after_masking = len(clear_specified_passes.time)
        #Turn any pixel in the frequency array with a value greater than 0.2 into a pixel of value 1
        #if the pixel value is 0.2 or lower it gets value 0
        is_water = np.where((frequency_array > 0.2),1,0) #has to be water in more than 20% of images to count
        #give the 'frequency' xarray back its new values of zero and one
        frequency.values = is_water
        #sum up the pixels
        number_water_pixels = frequency.sum(dim=['x', 'y'])
        #get the number
        number_water_pixels = number_water_pixels.values.tolist()
        #multiply by pixel size to get area in m2
        area_m2 = number_water_pixels*(25*25)

        surface_area_list.append([ID_caller, i, area_m2, n_images_used, n_images_after_masking])
        #print('This is the area as calculated from wet pixels at', i, 'meters', area_m2)

        #Plotting the image
        if make_plots:
            frequency.plot(figsize = (7,5))
    del wofs_albers
    del specified_passes
    del cc
    del clear_specified_passes
    del wet
    del dry
    del clear
    del frequency
    #delete the images when you finish each reservoir (otherwise the memory will run out and the kernel will break)
    return surface_area_list

## Run the function for all of the reservoirs
Now we will loop over all of the gauge data in 00_Library, and for each gauge we will apply the function which calculates the surface area for each depth interval. This will take over an hour if you left the number of passes uncapped. It will take less if you capped the passes.

In [5]:
array_list = []


def listsplit(N, K=1):
    length = len(N)
    return [N[i*length/K: (i+1)*length/K] for i in range(K)]


for ID in tqdm(ID_list, total=len(ID_list)):
    print("Working on gauge ", ID)
    if (ID in data_dict.keys()) and (ID in results.keys()):
        data = image_prod(ID, data_dict[ID], results[ID], make_plots = False)
        array_list.append(data)
        
        del data
    else:
        print('we didnt find', ID)        

  0%|          | 0/173 [00:00<?, ?it/s]

Working on gauge  ODSS_18335_WSLAHD.1


  0%|          | 0/9 [00:00<?, ?it/s]

Working on gauge  604.1


  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Working on gauge  M316


  0%|          | 0/6 [00:00<?, ?it/s]

Working on gauge  410543


  0%|          | 0/20 [00:00<?, ?it/s]

## Make a look up table of depth to surface area for each gauge ID

In [8]:
look_up_table = []
for i in array_list:
    df = DataFrame(i, columns = ['ID', 'Depth', 'Surface Area', 'number of images before masking', 'number of images after masking'])
    look_up_table.append(df)

df = pd.concat(look_up_table)
df = df.set_index("ID")  
df = df.drop(columns=['number of images before masking'])
df

Depth  Surface Area  number of images after masking
ID                                                                      
ODSS_18335_WSLAHD.1    220        231250                               5
ODSS_18335_WSLAHD.1    219        219375                              11
ODSS_18335_WSLAHD.1    218        206875                               8
ODSS_18335_WSLAHD.1    217        190625                               6
ODSS_18335_WSLAHD.1    216        170000                              10
...                    ...           ...                             ...
410543                1220       9996875                               7
410543                1221      10996250                               8
410543                1222      11526875                               3
410543                1223             0                               0
410543                1224      13356875                               4

[2572 rows x 3 columns]

## save the look up table as a csv file

In [ ]:
df.to_csv('2depth_to_surface_unlimitedImages.csv')